# Evolver Loop 2: Training Dynamics Analysis

## Goal
Analyze why ResNet50 with fine-tuning and TTA only achieved 2.4% improvement instead of expected 30-40%.

## Key Questions
1. Are the training dynamics healthy (convergence, overfitting, stability)?
2. Is the optimization configuration appropriate (LR, schedule, batch size)?
3. Are there data loading or augmentation issues?
4. What hyperparameters need tuning?

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Load session state to understand experiment history
with open('/home/code/session_state.json', 'r') as f:
    session_state = json.load(f)

print("=== EXPERIMENT SUMMARY ===")
print(f"Total experiments: {len(session_state['experiments'])}")
print(f"Best CV score: {min([exp['score'] for exp in session_state['experiments']]):.4f}")
print(f"Baseline (exp_000): 0.0736")
print(f"Best (exp_006): 0.0718")
print(f"Improvement: {(0.0736 - 0.0718) / 0.0736 * 100:.1f}%")
print(f"Gap to gold: {0.0718 - 0.038820:.4f}")
print(f"Improvement needed: {(0.0718 - 0.038820) / 0.0718 * 100:.1f}%")